In [3]:
!pip install datasets
!pip install -U sentence-transformers

In [4]:
import json
import math
import pandas as pd
from datasets import load_dataset,Dataset,DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, mean_absolute_percentage_error, r2_score, roc_curve
from sentence_transformers import SentenceTransformer, InputExample, losses, util, evaluation
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader

In [5]:
def ConvertJsonToList(fileName, split):
  subject1_fileDataset = load_dataset("json", data_files=fileName, split="train")
  samples = []

  for i in range (0,len(subject1_fileDataset[split][0])): #len(subject1)
    hashed_id = subject1_fileDataset[split][0][i]['hashed_id']
    mark = subject1_fileDataset[split][0][i]['mark']
    responseStudent = subject1_fileDataset[split][0][i]['responseStudent']
    responseTeacher = subject1_fileDataset[split][0][i]['responseTeacher']
    
    ie = InputExample(guid= hashed_id, texts=[responseTeacher, responseStudent], label=mark)
    samples.append(ie)

  return samples    

In [7]:
#Creating a list with the necesarry fields
data_test = ConvertJsonToList("Downloads/Prueba1/JoinedSubjects2.json", "test")

Using custom data configuration default-10fe12b0751c112a


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to C:\Users\javier.sanz\.cache\huggingface\datasets\json\default-10fe12b0751c112a\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


In [17]:
def TestModel(checkpoint, data, epochs, df):
  #Load model
  #local_model_path = 'jfarray' + '/' + checkpoint  + '_' + str(epochs) + '_Epochs'

  
  local_model_path = 'Downloads/Prueba1/1epoch' + '/' + checkpoint + '/' + str(epochs) + '_Epochs'

  model = SentenceTransformer(local_model_path)

  hashed_ids = []
  sentences1 = []
  sentences2 = []
  marks = []
  scores = []
  marksFloat = []
  scoresFloat = []

  for i in range (0,len(data)): #len(data)
    sentences1.append(data[i].texts[0])
    sentences2.append(data[i].texts[1])

  #Compute embedding for both lists
  embeddings1 = model.encode(sentences1, convert_to_tensor=True)
  embeddings2 = model.encode(sentences2, convert_to_tensor=True)
  
  #Compute cosine-similarits
  cosine_scores = util.cos_sim(embeddings1, embeddings2)

  #Output the pairs with their score
  for i in range(len(sentences1)):
    hashed_ids.append(data[i].guid)
    marks.append(str(data[i].label).replace('.',','))
    marksFloat.append(data[i].label)
    scores.append(str(round(cosine_scores[i][i].item(),3)).replace('.',','))
    scoresFloat.append(round(cosine_scores[i][i].item(),3))
  
  #Save scores in the file
  df['Hashed_id'] = hashed_ids
  df['Mark'] = marks
  df['Score_' + str(epochs)] = scores
  df['Sentence1'] = sentences1
  df['Sentence2'] = sentences2

  #Calculate metrics 'MAE', 'MSE', 'RMSE', 'RMSLE', 'MAPE', 'R2'
  metricMAE.append(str(mean_absolute_error(marksFloat, scoresFloat)).replace('.',','))
  metricMSE.append(str(mean_squared_error(marksFloat, scoresFloat, squared = True)).replace('.',','))
  metricRMSE.append(str(mean_squared_error(marksFloat, scoresFloat, squared = False)).replace('.',','))
  try:
    metricRMSLE.append(str(mean_squared_log_error(marksFloat, scoresFloat)).replace('.',','))
  except:
     metricRMSLE.append('-')
  metricMAPE.append(str(mean_absolute_percentage_error(marksFloat, scoresFloat)).replace('.',','))
  metricR2.append(str(r2_score(marksFloat, scoresFloat)).replace('.',','))

 #Evaluate Model this test data
  batch_size = 15 #Initializes the batch size with the same value as the training
  test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(data_test, batch_size=batch_size, name= checkpoint)
  test_evaluator(model, output_path= 'Downloads/Prueba1/1epoch/test/')
  


In [18]:
modelsToTest = [ 'distiluse-base-multilingual-cased-v1'
                ,'paraphrase-multilingual-MiniLM-L12-v2'
                ,'paraphrase-multilingual-mpnet-base-v2'
                ,'all-distilroberta-v1'
                ,'bert-base-multilingual-uncased'
                ,'dccuchile/bert-base-spanish-wwm-uncased'
              ]
#epochsToTest = [1,5,10,30,50,100]

#modelsToTest = [ 'distiluse-base-multilingual-cased-v1']
epochsToTest = [1]



#Train & Test the model
for checkpoint in modelsToTest:
  metricMAE = []
  metricMSE = []
  metricRMSE = []
  metricRMSLE = []
  metricMAPE = []
  metricR2 = []
  checkpoint = 'Model_' + checkpoint.replace('/','_')
  df = pd.DataFrame(columns=['Sentence1', 'Sentence2', 'Hashed_id', 'Mark'])
  dfMetrics = pd.DataFrame(columns=['Epochs', 'MAE', 'MSE', 'RMSE', 'RMSLE', 'MAPE', 'R2'])

  for epochs in epochsToTest:
    TestModel(checkpoint, data_test, epochs, df)

  #Save Score Results file
  df.to_csv('Downloads/Prueba1/1epoch/test/' + checkpoint +'_Scores_Results.csv', index=False, sep=';', encoding='utf-8')

  #Save Metrics file
  dfMetrics['Epochs'] = epochsToTest
  dfMetrics['MAE'] = metricMAE
  dfMetrics['MSE'] = metricMSE
  dfMetrics['RMSE'] = metricRMSE
  dfMetrics['RMSLE'] = metricRMSLE
  dfMetrics['MAPE'] = metricMAPE
  dfMetrics['R2'] = metricR2
  dfMetrics.to_csv('Downloads/Prueba1/1epoch/test/' +checkpoint +'_Metrics_Results.csv', index=False, sep=';', encoding='utf-8')

ValueError: ignored